In [ ]:
# To be able to make edits to repo without having to restart notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy import signal
from scipy import ndimage
from math import ceil
import cv2
import ot
import itertools

PROJECT_PATH = os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd())))
# PROJECT_PATH = os.getcwd()
sys.path.append(os.path.dirname(PROJECT_PATH))

from _prototypes.cell_remapping.src.remapping import pot_sliced_wasserstein
from _prototypes.cell_remapping.src.wasserstein_distance import _get_ratemap_bucket_midpoints
from library.map_utils import _temp_occupancy_map, _temp_spike_map, _speed2D, _speed_bins, _interpolate_matrix
from core.spatial import Position2D

unit_matcher_path = os.getcwd()
prototype_path = os.path.abspath(os.path.join(unit_matcher_path, os.pardir))
project_path = os.path.abspath(os.path.join(prototype_path, os.pardir))
lab_path = os.path.abspath(os.path.join(project_path, os.pardir))
sys.path.append(project_path)
os.chdir(project_path)
print(project_path)

In [ ]:
source = 'https://data.mendeley.com/datasets/hntn6m2pgk/1'
paper = 'https://www.cell.com/neuron/pdf/S0896-6273(21)00504-3.pdf'
supplementary = 'https://www.cell.com/cms/10.1016/j.neuron.2021.07.005/attachment/6428714e-08e0-46c4-bf9a-43697b565dde/mmc1'

# animal_id = 'Seattle_1006_1'
animal_id = 'Mumbai_1201_1'
# animal_id = 'Calais_0713_2'
# animal_id = 'Pisa_0502_1'

MEC_FRtensor = r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\Low et al\Data\{}_MEC_FRtensor.npy".format(animal_id)
MEC_spikes = r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\Low et al\Data\{}_MEC_spikes.npy".format(animal_id)
MEC_cellIDs = r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\Low et al\Data\{}_MEC_cellIDs.npy".format(animal_id)
MEC_behavior = r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\Low et al\Data\{}_behavior.npy".format(animal_id)

firing_rate = np.load(MEC_FRtensor)
spikes = np.load(MEC_spikes)
cellIDs = np.load(MEC_cellIDs)
behavior = np.load(MEC_behavior)



In [ ]:
idx = 0
cell = firing_rate[:,:,idx]
rate_map = firing_rate[:,:,idx]

In [ ]:
x = behavior[:,0]
speed = behavior[:,1]
y = behavior[:,2]
time = behavior[:,3]

In [ ]:
plt.imshow(rate_map)

In [ ]:
step = 1
window = 1
emds = []

for j in range(firing_rate.shape[2]):
    prev_slice = None
    rate_map = firing_rate[:,:,j]
    cell_emds = []
    print(j)
    for i in range(0, rate_map.shape[1],step):

        curr_slice = rate_map[:,i:i+window]

        if prev_slice is None:
            prev_slice = curr_slice

        y, x = curr_slice.shape
        # height_bucket_midpoints, width_bucket_midpoints = _get_ratemap_bucket_midpoints(([window],[400]), y, x)
        buckets = np.array(list(itertools.product(np.arange(0,y,1),np.arange(0,x,1))))
        source_weights = np.array(list(map(lambda x: prev_slice[x[0],x[1]], buckets)))
        target_weights = np.array(list(map(lambda x: curr_slice[x[0],x[1]], buckets)))
        source_weights = source_weights / np.sum(source_weights)
        target_weights = target_weights / np.sum(target_weights)
        # coord_buckets = np.array(list(itertools.product(height_bucket_midpoints,width_bucket_midpoints)))

        emd = ot.wasserstein_1d(buckets[:,0], buckets[:,0], source_weights, target_weights)
        # , n_projections=10**3)
        cell_emds.append(emd)

        prev_slice = curr_slice

    emds.append(cell_emds)

In [ ]:
fig = plt.figure(figsize=(10,10))

# to_plot = np.sort(np.array(emds)[:], axis=0)
to_plot = np.array(emds)[np.argsort(np.mean(np.array(emds), axis=1))]
plt.imshow(to_plot, cmap='jet')
plt.vlines(16,0,len(to_plot),color='white')
plt.vlines(16*2,0,len(to_plot),color='white')
plt.vlines(16*3,0,len(to_plot),color='white')
plt.vlines(16*4,0,len(to_plot),color='white')
plt.vlines(16*5,0,len(to_plot),color='white')
plt.ylim(len(to_plot),250)
plt.xlim(0,80)

In [ ]:
to_plot.shape